In [28]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

In [29]:
nodes = pd.read_csv('Nodes with attributes.csv')

In [30]:
link = pd.read_csv('Links - Intragroup.csv')

In [31]:
nodes.columns

Index(['ID', 'Full Name', 'Invited', 'Responded', 'Organization', 'Division',
       'Function/Deparment', 'Title', 'Level', 'Location', 'Tenure', 'Manager',
       'Team', 'Gender'],
      dtype='object')

In [32]:
nodes['ID'].nunique()

612

In [33]:
nodes = nodes[['ID','Gender']]

In [34]:
nodes

,ID,Gender
0,428634,Female
1,428764,Female
2,428881,Female
3,428738,Female
4,428803,Female
...,...,...
607,428244,Female
608,428192,Male
609,428231,Male
610,442219,Male


In [35]:
link

,Source,Target,Label
0,427685,428855,Energy
1,428309,428686,Trust
2,428309,428686,Career Development
3,428309,428686,Personal Support
4,428309,428686,Decision Making
...,...,...,...
5088,427919,428049,Access
5089,427919,428049,Personal Support
5090,427919,428049,Decision Making
5091,427932,427958,Trust


In [36]:
link_source = link.rename(columns = {'Source': "ID"})

In [37]:
link_source

,ID,Target,Label
0,427685,428855,Energy
1,428309,428686,Trust
2,428309,428686,Career Development
3,428309,428686,Personal Support
4,428309,428686,Decision Making
...,...,...,...
5088,427919,428049,Access
5089,427919,428049,Personal Support
5090,427919,428049,Decision Making
5091,427932,427958,Trust


In [38]:
out_degree = pd.merge(link_source,nodes,how='left',on = 'ID')

In [39]:
out_degree

,ID,Target,Label,Gender
0,427685,428855,Energy,Female
1,428309,428686,Trust,Male
2,428309,428686,Career Development,Male
3,428309,428686,Personal Support,Male
4,428309,428686,Decision Making,Male
...,...,...,...,...
5088,427919,428049,Access,Male
5089,427919,428049,Personal Support,Male
5090,427919,428049,Decision Making,Male
5091,427932,427958,Trust,Male


In [46]:
by_label_out_degree = out_degree.groupby(['Gender','Label']).ID.count()

In [47]:
by_label_out_degree

Gender  Label             
Female  Access                 96
        Career Development    165
        Decision Making       230
        Energy                321
        Innovation            220
        Personal Support      234
        Problem Solving       298
        Strategy              196
        Trust                 274
Male    Access                133
        Career Development    236
        Decision Making       379
        Energy                445
        Innovation            376
        Personal Support      260
        Problem Solving       451
        Strategy              349
        Trust                 430
Name: ID, dtype: int64

In [51]:
link_source2 = link.rename(columns = {"Target":"ID"})

In [56]:
link_source2

,Source,ID,Label
0,427685,428855,Energy
1,428309,428686,Trust
2,428309,428686,Career Development
3,428309,428686,Personal Support
4,428309,428686,Decision Making
...,...,...,...
5088,427919,428049,Access
5089,427919,428049,Personal Support
5090,427919,428049,Decision Making
5091,427932,427958,Trust


In [57]:
in_degree = pd.merge(link_source2,nodes,how='left',on = 'ID')

In [58]:
in_degree

,Source,ID,Label,Gender
0,427685,428855,Energy,Female
1,428309,428686,Trust,Male
2,428309,428686,Career Development,Male
3,428309,428686,Personal Support,Male
4,428309,428686,Decision Making,Male
...,...,...,...,...
5088,427919,428049,Access,Male
5089,427919,428049,Personal Support,Male
5090,427919,428049,Decision Making,Male
5091,427932,427958,Trust,Male


In [61]:
by_label_in_degree = in_degree.groupby(['Gender','Label']).ID.count()

In [62]:
by_label_in_degree

Gender  Label             
Female  Access                 62
        Career Development    135
        Decision Making       199
        Energy                308
        Innovation            198
        Personal Support      213
        Problem Solving       247
        Strategy              117
        Trust                 237
Male    Access                167
        Career Development    266
        Decision Making       410
        Energy                458
        Innovation            398
        Personal Support      281
        Problem Solving       502
        Strategy              428
        Trust                 467
Name: ID, dtype: int64